In [1]:
### Function that takes in a dataframe, removes outliers, problably normalize and scale data
# Functions are set up in such a way where we should be able to batch pre-process the data without reading in the entire data set at once

In [2]:
import pandas as pd
import numpy as np

In [3]:
# Set types
# https://gis.stackexchange.com/questions/8650/measuring-accuracy-of-latitude-and-longitude/8674#8674
# Floats have 7 decimal digits of precision, doubles 15. You only need around 7 for lat and long 
types_dict = {
        "fare_amount": np.float16,
        "pickup_longitude": np.float32, 
        "pickup_latitude": np.float32, 
        "dropoff_longitude": np.float32, 
        "dropoff_latitude": np.float32, 
        "passenger_count": np.int8,
        "pickup_datetime": np.int8,
}

In [4]:
def _remove_outliers(df):
    """Takes in an unprocessed df and returns a df that has outliers removed"""
    ## Remove obvious outliers first
    # where lat/long == 0
    df = df.drop(df.index[df['pickup_longitude'] == 0])
    df = df.drop(df.index[df['dropoff_longitude'] == 0])
    # Where fares < 0, becuase that makes no sense
    df = df.drop(df.index[df['fare_amount'] < 0])
    # Realistically, if we want to keep the trips within manhattan, pickup long should be between -72 and -75 and lat between 40 and 42
    # as everything else is an extreme outlier
    df = df.drop(df.index[df['pickup_longitude'] > -72])
    df = df.drop(df.index[df['pickup_longitude'] < -75])
    df = df.drop(df.index[df['pickup_latitude'] > 42])
    df = df.drop(df.index[df['pickup_latitude'] < 40])
    # Visualizations show the same thing with dropof longs and lats
    df = df.drop(df.index[df['dropoff_longitude'] > -72])
    df = df.drop(df.index[df['dropoff_longitude'] < -75])
    df = df.drop(df.index[df['dropoff_latitude'] > 42])
    df = df.drop(df.index[df['dropoff_latitude'] < 40])
    return df

In [5]:
# we're assuming that a df of the right types is being passed in
def preprocess(df,types=types_dict):
    """Takes in an unprocessed df and returns a df that has outliers removed and is preprocessed"""
    # Drop NA values
    df = df.dropna()
    # Remove obvious outliers first
    df = _remove_outliers(df)
    # Key isnt useful
    del df["key"]
    # Currently giving categorial numbers, but something like onehotencoding might work better
    df["pickup_datetime"] = df["pickup_datetime"].apply(lambda d:int(d[10:13]))
    # Set types as we batch read in
    for name, t in types_dict.items():
        df[name] = df[name].astype(t)
    # TODO - Scale data. Requires us to find min and max of entire dataset, not sure how to proceed since we're batching
    # TODO - Normalize data, same issues as above and batching.
    # TODO - Possibly PCA, but seems like all of the features we get should be relevant to the task at hand
    return df

In [6]:
### test functions
chunksize = 10 ** 6
filename = "../data/train.csv"

dataset_df = pd.DataFrame()  # main dataset

from tqdm import tqdm
# Pretty progress bar
pbar = tqdm(total=len(range(54)))       
for index, df_chunk in enumerate(pd.read_csv(filename, chunksize=chunksize)):
    pbar.update(1)
    dataset_df = pd.concat([dataset_df, preprocess(df_chunk)])
pbar.close()

size_bytes = dataset_df.memory_usage(deep=True).sum()
print("Entire dataset fits in {:.2f} GB".format(size_bytes/1e+9))
print("Number of rows: {}".format(dataset_df.count()[0]))

100%|██████████| 54/54 [02:33<00:00,  2.99s/it]


Entire dataset fits in 1.48 GB
Number of rows: 52756015


In [7]:
fares = dataset_df["fare_amount"]
del dataset_df["fare_amount"]
from sklearn.externals import joblib

In [8]:
dataset_df.head()

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,17,-73.844315,40.721317,-73.841614,40.712276,1
1,16,-74.016045,40.711304,-73.979271,40.782005,1
2,0,-73.982735,40.761269,-73.991241,40.750561,2
3,4,-73.987129,40.733143,-73.991570,40.758091,1
4,7,-73.968094,40.768009,-73.956657,40.783764,1


In [9]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler().fit(dataset_df)
# then on new data call scaler.transform(data) to scale
joblib.dump(scaler, '../models/minmaxscaler.joblib') 
del scaler

/home/jovyan/.conda/envs/jovyan/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int8, float32 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [10]:
#  We need to normalize on scaled data I think, but I cant fit the scaled df into memory along with the original df
from sklearn.preprocessing import Normalizer
normalizer = Normalizer().fit(dataset_df)
joblib.dump(normalizer, '../models/normalizer.joblib') 
del normalizer